## We'll collect the data from kayak, a popular site for airline tickets.

In [13]:

import os
os.environ['DISPLAY'] = 'host.docker.internal:1.0'



In [1]:
import pandas as pd
import datetime
from bs4 import BeautifulSoup
from time import sleep


In [ ]:
from playwright.async_api import async_playwright

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # 
        page = await browser.new_page()
    
     
await main()


TargetClosedError: Page.wait_for_timeout: Target page, context or browser has been closed

### Extract the air companies,total escapes,price and vol delay

In [53]:
def get_airlines(soup):
    airline = []
    airlines = soup.find_all('div', class_='J0g6-operator-text')
    for line in airlines:
        text = line.text
        airline.append(text.strip())
    return airline

def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('span', class_='JWEO-stops-text')
    for i in stops:
        text = i.text.lower()
        if text == "direct":
            stops_list.append("0")
        else:
            stops_list.append(text.split(" ")[0].strip())
    return stops_list

def get_price(soup):
    price_list = []
    prices = soup.find_all('div', class_='f8F1-price-text')
    for i in prices:
        price_list.append(i.text.split("&")[0].strip())  # Nettoyage du texte
    return price_list

def get_duration(soup):
    duration_list = []
    durations = soup.find_all('div', class_='xdW8')
    for i in durations:
        duration = i.find("div", class_='vmXl')
        if duration:
            duration_list.append(duration.text.strip())  # Ajout uniquement si trouvé
    return duration_list


### Start scrapping

In [83]:
depart="TUN"
Destinations=["MAD","COO","CRL","ANR","OST","LUX","NDJ","ABJ","PAR","ALG"]

# MAD : Madrid
# COO : Cotonou
# TUN : Tunis
# CRL : Brussels: Belgique
# ANR : Antwerp :Belgique
# OST : Ostrava :Belgique
# LUX : Luxembourg 
# NDJ : N'Djamena : Tchad
#ABJ :Abidjan :Cote d'ivoire
#PAR : Paris :France
#ALG : Alger :Algerie


# Définir la date de départ
# date_a = pd.to_datetime("01/12/2024", dayfirst=True)
date_a = pd.to_datetime("19/07/2025", dayfirst=True)

# Définir la date de fin, ici pour une année complète (365 jours)
end_date = date_a + datetime.timedelta(days=365)


# Résultats
result = pd.DataFrame()

# Fonction principale pour scraper les données
async def main ():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # Changez headless=True si nécessaire
        page = await browser.new_page()
        current_date = date_a
        while current_date < end_date:
            date_time_a = current_date.strftime("%Y-%m-%d")

            for destination in Destinations:
                if await page.locator("text=Êtes-vous humain ?").is_visible():
                    print("Captcha détecté. Veuillez le résoudre manuellement.")

                    input("Après avoir résolu le Captcha, appuyez sur Entrée pour continuer...")
                
                print(f"Scraping {depart} -> {destination} for {date_time_a} ")
                url = f"https://www.kayak.fr/flights/{depart}-{destination}/{date_time_a}?ucs=12uvb0c&sort=bestflight_a"
                
                await page.goto(url)
                
               
                await page.wait_for_timeout(25000)

                # Gestion du bouton "Show more" (si présent)
                try:
                    show_more_button = page.locator('.show-more-button')
                    if await show_more_button.is_visible():
                        await show_more_button.click()
                        await page.wait_for_timeout(25000)  # Attente après clic
                except Exception as e:
                    print(f"Problème avec le bouton 'Show more' : {e}")
                    await input("Résolvez le captcha manuellement, puis appuyez sur Entrée pour continuer...")

                # Extraction des données avec BeautifulSoup
                html = await page.content()
                soup = BeautifulSoup(html, "html.parser")

                # Parsing des données
                airlines = get_airlines(soup)
                total_stops = get_total_stops(soup)
                prices = get_price(soup)
                durations = get_duration(soup)
                print(f"Airlines: {len(airlines)}, Total Stops: {len(total_stops)}, Prices: {len(prices)}, Durations: {len(durations)}")

                # Création du DataFrame pour cette itération
                df = pd.DataFrame({
                    "Airlines": airlines,
                    "Total_Stops": total_stops,
                    "Price": prices,
                    "Duration": durations,
                    "Destination": destination,
                    "Date": date_time_a,
                }, columns=["Airlines", "Total_Stops", "Price", "Duration", "Destination", "Date"])

                # Ajout des données au résultat global
                global result
                result = pd.concat([result, df], ignore_index=True)

                current_date += datetime.timedelta(days=1)

        await browser.close()

# Exécution du script
await main()



Scraping TUN -> MAD for 2025-07-19 
Airlines: 0, Total Stops: 0, Prices: 0, Durations: 0
Captcha détecté. Veuillez le résoudre manuellement.
Scraping TUN -> COO for 2025-07-19 


TargetClosedError: Page.goto: Target page, context or browser has been closed

#### Extration 

In [82]:
# Sauvegarde des résultats dans un fichier CSV
result.to_csv("flights_data7.csv", index=False)
print("Données sauvegardées dans flights_data.csv")


Données sauvegardées dans flights_data.csv
